In [1]:
import numpy as np
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
def resub_(words_):
    words_ = re.sub(r'\W', ' ', words_)
    words_ = re.sub(r'\d', ' ', words_)
    words_ = re.sub(r'\b\w{0,2}\b', ' ', words_)
    words_ = re.sub('\s+', ' ', words_)
    return words_

In [3]:
def get_headings(soup):
    tag_title_ = str()
    tag_h1_ = str()
    tag_h2_ = str()
    tag_h3_ = str()
    tag_a_ = str()
    
    for title in soup.find_all(['title']):
        tag_title_ += (title.text + ' ')
        
    for h1 in soup.find_all(['h1']):
        tag_h1_ += (h1.text + ' ')

    for h2 in soup.find_all(['h2']):
        tag_h2_ += (h2.text + ' ')

    for h3 in soup.find_all(['h3']):
        tag_h3_ += (h3.text + ' ')

    for a in soup.find_all(['a']):
        tag_a_ += (a.text + ' ')
    
    tag_title_ = resub_(tag_title_)
    tag_h1_ = resub_(tag_h1_)
    tag_h2_ = resub_(tag_h2_)
    tag_h3_ = resub_(tag_h3_)
    tag_a_ = resub_(tag_a_)
    

    return tag_title_, tag_h1_, tag_h2_, tag_h3_, tag_a_ 

In [4]:
#Список имеет следующий вид: list = [text_1, text2, ... , textn]
def tags_of_group(docs_id_):
    
    tag_title_l = list()
    tag_h1_l = list()
    tag_h2_l = list()
    tag_h3_l = list()
    tag_a_l = list()
    
    for i in docs_id_:
        
        file_ = open('./content/' + str(i) + '.dat', 'r', encoding = 'utf-8')
        text_ = file_.read()
        soup_ = BeautifulSoup(text_, 'html')
        tag_title_, tag_h1_, tag_h2_, tag_h3_, tag_a_ = get_headings(soup_)
        tag_title_l.append(tag_title_)
        tag_h1_l.append(tag_h1_)
        tag_h2_l.append(tag_h2_)
        tag_h3_l.append(tag_h3_)
        tag_a_l.append(tag_a_)
        
    return tag_title_l, tag_h1_l, tag_h2_l, tag_h3_l, tag_a_l

In [81]:
def get_docs_id(group):
    
    d = pd.read_csv('./data/test_groups.csv')
    df_new = d[d.group_id == group]
    
    return df_new['doc_id']

In [64]:
def a_lot_of_time(start, fin):
    c = pd.DataFrame()
    for i in range(start, fin + 1):
        doc_id = get_docs_id(i)
        title_corp, h1_corp, h2_corp, h3_corp, a_corp = tags_of_group(get_docs_id(i))
        df = pd.DataFrame({'title'  : pd.Series(title_corp),
                           'h1' : pd.Series(h1_corp),
                           'h2' : pd.Series(h2_corp),
                           'h3' : pd.Series(h3_corp),
                           'a'  : pd.Series(a_corp),
                           'group_id' : i,
                           'doc_id' : list(doc_id)})
        c = pd.concat([c,df], ignore_index = True)
        print('group', i, 'has been added')
    return c

In [99]:
%%time
main_df = a_lot_of_time(130,309)

Wall time: 0 ns


In [97]:
main_df.to_csv('core_test.csv', encoding = 'utf8') #sep = ','